# Extract luminosity data from spreadsheet

- Spreadsheet location: https://docs.google.com/spreadsheets/d/10LLDj0NRvGVtQPhoFLRR2A7NRblJRvnX_U7wvfeYhvw/edit?pli=1&gid=93064947#gid=93064947
- Spreadsheet owner: prachurjya.hazarika.students.iiserpune.ac.in

In [1]:
import os, sys
import csv
import json
import pandas as pd

In [2]:
#Load csv file into a dataframe:
csv_file = 'Data and MC Sample Database - Luminosity.csv'
lumidata = pd.DataFrame()

colnames = [
    'sample', 
    'subsample', 
    '2016preVFP_UL', 
    '2016postVFP_UL', 
    '2017_UL', 
    '2018_UL', 
    'Run2'
]

#Read the csv file into a list:
with open(csv_file, newline='') as csvfile:
    reader = csv.reader(csvfile)
    lumilist = list(reader)

#print(lumilist)
lumidata = pd.DataFrame(lumilist)

#Dropping unnecessary columns:
lumidata = lumidata[[1, 2, 3, 4, 5, 6, 7]]
lumidata.columns = colnames

#Dropping a first few rows:
lumidata = lumidata.drop(index=range(8)).reset_index(drop=True)

# Dropping empty rows:
lumidata = lumidata[~lumidata.apply(lambda row: all(cell == '' for cell in row), axis=1)].reset_index(drop=True)

# Dropping rows where the first column has a value (e.g., process name) but all other columns are empty
lumidata = lumidata[~lumidata.apply(lambda row: row.iloc[0] != '' and all(cell == '' for cell in row.iloc[1:]), axis=1)].reset_index(drop=True)

# Replace empty strings with NaN and then forward fill
lumidata['sample'] = lumidata['sample'].replace('', pd.NA).ffill()

# Convert numerical columns to float, skipping 'sample' and 'subsample'
for col in colnames[2:]: lumidata[col] = pd.to_numeric(lumidata[col], errors='coerce')

# Display the updated DataFrame
display(lumidata.head(10))
display(lumidata.tail(10))

,sample,subsample,2016preVFP_UL,2016postVFP_UL,2017_UL,2018_UL,Run2
0,QCD_EMEnriched,15to20,3.0686,3.0410,6.0173,5.9667,18.0936
1,QCD_EMEnriched,20to30,1.4617,1.4573,2.8934,2.9266,8.7390
2,QCD_EMEnriched,30to50,0.6766,0.6749,1.3626,1.3300,4.0441
3,QCD_EMEnriched,50to80,2.7368,2.7384,5.1360,5.2803,15.8915
4,QCD_EMEnriched,80to120,13.1901,13.0743,27.7834,25.7643,79.8120
5,QCD_EMEnriched,120to170,72.8724,75.1967,148.7347,145.3357,442.1395
6,QCD_EMEnriched,170to300,111.6403,111.9813,221.3117,223.5043,668.4375
7,QCD_EMEnriched,300toInf,1035.1223,1031.4692,2006.2808,2007.2409,6080.1132
8,QCD_MuEnriched,20to30,12.5794,12.1566,25.4622,23.8930,74.0914
9,QCD_MuEnriched,30to50,20.9385,25.9125,42.8339,42.9055,132.5904


,sample,subsample,2016preVFP_UL,2016postVFP_UL,2017_UL,2018_UL,Run2
70,WWZ,Inclusive,4.745167e+05,3.925015e+05,1.042765e+06,1.452841e+06,3.362624e+06
71,WZZ,Inclusive,2.802592e+06,2.399720e+06,5.219828e+06,5.254861e+06,1.567700e+07
72,ZZZ,Inclusive,5.487805e+06,4.878049e+06,1.205962e+07,1.693767e+07,3.936314e+07
73,Higgs,bbH_HToZZTo4L,3.603684e+09,3.442003e+09,3.783024e+09,3.533384e+09,1.436210e+10
74,Higgs,GluGluHToZZTo4L,3.052503e+10,3.052503e+10,3.046398e+10,2.869353e+10,1.202076e+11
75,Higgs,GluGluToZH,7.647487e+07,7.708088e+07,7.487232e+07,7.555436e+07,3.039824e+08
76,Higgs,GluGluZH,9.441088e+07,9.148640e+07,3.561167e+09,3.444441e+08,4.091508e+09
77,Higgs,ttHToNonbb,9.338980e+06,4.348418e+06,2.394235e+07,3.460337e+07,7.223311e+07
78,Higgs,VBF_HToZZTo4L,1.861386e+08,4.930693e+08,4.940594e+08,4.722772e+08,1.645545e+09
79,Higgs,VHToNonbb,4.611703e+06,3.600981e+06,9.475417e+06,1.316648e+07,3.085458e+07


In [3]:
os.makedirs('jsondata', exist_ok=True)

signaldict = {
    "VLLS_ele":{
        "M100":     512598.29,
        "M125":    1091218.35,
        "M150":    2064227.59,
        "M200":    5696971.43,
        "M250":   10804152.17,
        "M300":   21769257.64,
        "M350":   15935520.00,
        "M400":   27645555.56,
        "M450":   44700229.36,
        "M500":   72181021.90,
        "M750":  254269230.77,
        "M1000":1268884615.38
    },
    "VLLS_mu":{
        "M100":     508329.91,
        "M125":    1100348.62,
        "M150":    2064344.83,
        "M200":    5710047.62,
        "M250":   10794347.83,
        "M300":   21809519.65,
        "M400":   27595277.78,
        "M450":   45337844.04,
        "M500":   72847810.22,
        "M750":  255569230.77,
        "M1000":1275589743.59
    },
    "VLLD_ele":{
        "M100":     6560.41,
        "M200":    54213.24,
        "M300":    85061.86,
        "M400":   270022.05,
        "M600":  1651744.97,
        "M800":  6824207.49,
        "M1000":25011328.53
    },
    "VLLD_mu":{
        "M100":   6622.84,
        "M200":  54344.12,
        "M300":  85986.25,
        "M400": 267905.18,
        "M600":1670469.80,
        "M800":7136311.24
    },
    "VLLS_tau":{
        "M100":    1164480.52,
        "M125":    2232460.95,
        "M150":    3928735.63,
        "M200":   10246631.84,
        "M250":   18374444.97,
        "M300":   36726747.17,
        "M350":   26462975.32,
        "M400":   45822297.89
    }
}

datadict_2018 = {
    "SingleMuon":{"SingleMuon_A":59830,"SingleMuon_B":59830,"SingleMuon_C":59830,"SingleMuon_D":59830},
    "EGamma":{"EGamma_A":59830,"EGamma_B":59830,"EGamma_C":59830,"EGamma_D":59830}
}
datadict_2017 = {
    "SingleMuon":{"SingleMuon_B": 41480,"SingleMuon_C": 41480,"SingleMuon_D": 41480,"SingleMuon_E": 41480,"SingleMuon_F": 41480},
    "EGamma":{"EGamma_B": 41480,"EGamma_C": 41480,"EGamma_D": 41480,"EGamma_E": 41480,"EGamma_F": 41480}
}
datadict_2016postVFP ={
    "SingleMuon":{"SingleMuon_F": 16227,"SingleMuon_G": 16227,"SingleMuon_H": 16227},
    "EGamma":{"EGamma_F": 16227,"EGamma_G": 16227,"EGamma_H": 16227}
}
datadict_2016preVFP = {
    "SingleMuon":{"SingleMuon_B": 19692,"SingleMuon_C": 19692,"SingleMuon_D": 19692,"SingleMuon_E": 19692,"SingleMuon_F": 19692},
    "EGamma":{"EGamma_B": 19692,"EGamma_C": 19692,"EGamma_D": 19692,"EGamma_E": 19692,"EGamma_F": 19692}
}

def dict_to_dataframe(data_dict, campaign):
        data_list = []
        for sample, subsamples in data_dict.items():
            for subsample, lumi in subsamples.items():
                data_list.append([sample, subsample, lumi])
        
        df = pd.DataFrame(data_list, columns=["sample", "subsample", campaign])
        return df

def write_lumidata_into_json(lumidata, campaign):

    if campaign not in lumidata.columns:  raise ValueError(f"Campaign name '{campaign}' does not exist as a column in the data.")
        
    df_signal = dict_to_dataframe(signaldict, campaign)
    df_data = pd.DataFrame()
    if   '2018'        in campaign: df_data = dict_to_dataframe(datadict_2018,        campaign)
    elif '2017'        in campaign: df_data = dict_to_dataframe(datadict_2017,        campaign)
    elif '2016postVFP' in campaign: df_data = dict_to_dataframe(datadict_2016postVFP, campaign)
    elif '2016preVFP'  in campaign: df_data = dict_to_dataframe(datadict_2016preVFP,  campaign)
    
    lumidata_combined = pd.concat([lumidata, df_signal, df_data], ignore_index=True)
    
    campaign_data = {}
    
    grouped_data = lumidata_combined.groupby('sample')
    for sample, group in grouped_data:
        campaign_data[sample] = {}

        for _, row in group.iterrows():
            subsample = row['subsample']
            campaign_data[sample][subsample] = row[campaign]

    ### Sort accroding to sample name:
    campaign_data = dict(sorted(campaign_data.items()))

    ### Push signal and data towards the end of the dictionary.
    end_samples = ['VLLD_ele', 'VLLD_mu', 'VLLS_ele', 'VLLS_mu', 'VLLS_tau', 'SingleMuon', 'EGamma', 'EGamma']
    
    ### Move the signal and data entries to the end
    for sample in end_samples:
        if sample in campaign_data:
            campaign_data[sample] = campaign_data.pop(sample)
    
    ### Write the dictionary to a JSON file
    outfile = os.path.join('jsondata',f'lumidata_{campaign}.json')
    with open(outfile, 'w') as json_file: json.dump(campaign_data, json_file, indent=4)
    print('File written: '+outfile)

print('Ready to write!')

Ready to write!


In [4]:
write_lumidata_into_json(lumidata, "2016preVFP_UL")
write_lumidata_into_json(lumidata, "2016postVFP_UL")
write_lumidata_into_json(lumidata, "2017_UL")
write_lumidata_into_json(lumidata, "2018_UL")
write_lumidata_into_json(lumidata, "Run2")

File written: jsondata/lumidata_2016preVFP_UL.json
File written: jsondata/lumidata_2016postVFP_UL.json
File written: jsondata/lumidata_2017_UL.json
File written: jsondata/lumidata_2018_UL.json
File written: jsondata/lumidata_Run2.json
